## Importação de bibliotecas e dados externos

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
shark_attack = pd.read_csv('archive.zip', compression='zip', encoding='ansi')
shark_attack.tail()

### Verificando DataFrame

In [ ]:
shark_attack.info()

In [ ]:
# aumentando a quantidade de colunas do dataframe a serem visualizadas

pd.options.display.max_columns = 25

In [ ]:
# descobrindo erros nos nomes das colunas

shark_attack.columns

##  Tratamento de linhas nulas

In [ ]:
# criando uma coluna com a contagem de nulos em cada linha

shark_attack['Vazio'] = shark_attack.T.isnull().sum()

In [ ]:
shark_attack.head()

In [ ]:
shark_attack['Vazio'].value_counts()

In [ ]:
# criando um dataframe com todas as linhas que possuem menos de 20 colunas vazias

f_shark = shark_attack.drop(shark_attack[shark_attack['Vazio'] > 20].index)

In [ ]:
# verificando a quantidade de linhas x colunas vazias

f_shark['Vazio'].value_counts()

##  Escolha de colunas relevantes

In [ ]:
f_shark['Unnamed: 23'].value_counts()

In [ ]:
f_shark.loc[~f_shark['Unnamed: 23'].isnull(), :]

In [ ]:
# escolhendo as colunas relevantes

f_shark = f_shark[['Date', 'Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury',
             'Fatal (Y/N)', 'Time', 'Species ']]

In [ ]:
# renomeando as colunas

f_shark.columns = ['date', 'year', 'type', 'country', 'area', 'location', 'activity', 'name', 'sex', 'age', 'injury',
                 'fatal', 'time', 'species']

In [ ]:
f_shark.tail()

## Tratamento de ano

In [ ]:
# verificando linhas cujo ano pode ser completado com informação da coluna de data

f_shark.loc[~(f_shark['date']).isnull() & (f_shark['year'].isnull()), 'date']

In [ ]:
# preenchendo valor de ano com informação de data

f_shark.loc[~(f_shark['date']).isnull() & (f_shark['year'].isnull()), 'year'] = f_shark.loc[~(f_shark['date']).isnull() & (f_shark['year'].isnull()), 'date'].str.slice(start=-4)

In [ ]:
f_shark.loc[~(f_shark['date']).isnull() & (f_shark['year'].isnull()), 'date']

In [ ]:
# transformando coluna ano em inteiros

f_shark['year'] = f_shark['year'].astype(int)

In [ ]:
# verificando dados pós-tratamento

f_shark['year'].isnull().sum()

In [ ]:
f_shark.info()

## Tratamento de país

In [ ]:
f_shark['country']

In [ ]:
f_shark.loc[(f_shark['country'].isnull()) & (f_shark['area'].isnull()) & (f_shark['location'].isnull()), :]

In [ ]:
# verificando linhas sem informação de país

f_shark.loc[(f_shark['country'].isnull()) & (~(f_shark['area'].isnull()) | ~(f_shark['location'].isnull())), ['country', 'area', 'location']]

In [ ]:
# substituindo campos 'country' nulos por informações das colunas 'area' ou 'location'

f_shark.loc[(f_shark['country'].isnull()) & ((f_shark['area']=='St Helena') | ~(f_shark['area'].isnull())), 'country'] = f_shark.loc[(f_shark['country'].isnull()) & ((f_shark['area']=='St Helena') | ~(f_shark['area'].isnull())), 'area']
f_shark.loc[(f_shark['country'].isnull()) & ~(f_shark['location'].isnull()), 'country'] = f_shark.loc[(f_shark['country'].isnull()) & ~(f_shark['location'].isnull()), 'location']

In [ ]:
f_shark['country'].isnull().sum()

In [ ]:
# ajustando nomes de paises

f_shark.loc[~(f_shark['country'].isnull()), 'country'] = f_shark.loc[~(f_shark['country'].isnull()), 'country'].apply(lambda place: re.sub(' ?[\/\(].*', '', str(place)) if len(str(place))>0 else None)
f_shark.loc[~(f_shark['country'].isnull()), 'country'] = f_shark.loc[~(f_shark['country'].isnull()), 'country'].str.lstrip(' ')
f_shark.loc[~(f_shark['country'].isnull()), 'country'] = f_shark.loc[~(f_shark['country'].isnull()), 'country'].str.rstrip('[ ?]')

## Agrupado em dados de oceanos/mar

In [ ]:
f_shark['ocean']=f_shark['country']

In [ ]:
oceano = 'OCEAN|SEA|BAY|GULF'

lista = sorted(list(f_shark.loc[~(f_shark['ocean'].isnull()), 'ocean'].apply(lambda pais: pais if re.search(oceano, pais.upper()) else None).value_counts().index))
lista

In [ ]:
f_shark.loc[f_shark['ocean']=='PALESTINIAN TERRITORIES', :]

###  Filtrando oceanos pelas colunas oceano e área

In [ ]:
# substituindo locais do Oceano Atlântico

atl_keys = 'atlant|caribb|mediterr|north s'
filtro_atl = (f_shark['ocean'].apply(lambda pais: True if re.search(atl_keys, str(pais).lower()) else False)) | (f_shark['area'].apply(lambda pais: True if re.search(atl_keys, str(pais).lower()) else False))
f_shark.loc[filtro_atl, 'ocean'] = 'ATLANTIC OCEAN'

In [ ]:
# substituindo locais do Oceano Pacífico

pcf_keys = 'pacif'
filtro_pcf = (f_shark['ocean'].apply(lambda pais: True if re.search(pcf_keys, str(pais).lower()) else False)) | (f_shark['area'].apply(lambda pais: True if re.search(pcf_keys, str(pais).lower()) else False))
f_shark.loc[filtro_pcf, 'ocean'] = 'PACIFIC OCEAN'

In [ ]:
# substituindo locais do Oceano Índico

ind_keys = 'india|persian|red sea'
filtro_ind = (f_shark['ocean'].apply(lambda pais: True if re.search(ind_keys, str(pais).lower()) else False)) | (f_shark['area'].apply(lambda pais: True if re.search(ind_keys, str(pais).lower()) else False))
f_shark.loc[filtro_ind, 'ocean'] = 'INDIAN OCEAN'

###  Filtrando oceanos por países

In [ ]:
#filtrando por paises no Oceano Atlântico

pais_atl = 'carlisle|st helena|brazil|haiti|honduras|argentina|peru|ecuador|venezu|columbia|spain|sloven|norw|guinea|georgia|aruba|gabon|angol|belize|crete|cayman|grenada|dominic|liberia|nigeria|france|united ki|scotl|turkey|el salv|england|ireland|azor|icel|falkl|guyana|cape v|florida|namib|curacao|bahamas|roatan|antigu|bermud|portugal|ghana|sierra|thomas|barbados|cuba|jamaica|puerto|artin|tobago|caico|urug|ionian|greece|croatia|italy|israel|lebanon|malta|monaco|montenegro|balkan|tunisia|libya|syria|alger|cypr'

filtro_atl = f_shark['ocean'].apply(lambda pais: True if re.search(pais_atl, str(pais).lower()) else False)
f_shark.loc[filtro_atl, 'ocean'] = 'ATLANTIC OCEAN'

In [ ]:
#filtrando por paises no Oceano Pacífico

pais_pcf = 'china sea|tasman|^ocean|california|korea|japan|marshal|canada|kiribati|philippines|admir|tonga|asia|costa r|nicar|seneg|malay|singap|russia|mariana|hong kong|cook|johnston|solomon|guam|chile|fiji|microne|thail|palau|new brit|polyne|maldiv|zealand|samoa|okinawa|papua|new guinea|taiwan|tuvalu|vanuatu|caled|china|timor|vietn'

filtro_pcf = f_shark['ocean'].apply(lambda pais: True if re.search(pais_pcf, str(pais).lower()) else False)
f_shark.loc[filtro_pcf, 'ocean'] = 'PACIFIC OCEAN'

In [ ]:
#filtrando por paises no Oceano Índico

pais_ind = 'bengal|arabian sea|aden|madagascar|reunion|mozamb|andam|egypt|bahrein|diego g|saudi a|arab em|mayote|burma|comoros|yeme|indonesia|java|banglad|maurit|keny|djib|seyc|sri|ceyl|tanza|somal|kuwai|iran|iraq'

filtro_ind = f_shark['ocean'].apply(lambda pais: True if re.search(pais_ind, str(pais).lower()) else False)
f_shark.loc[filtro_ind, 'ocean'] = 'INDIAN OCEAN'

### Tratamento dados dos EUA 

In [ ]:
# filtrando dados dos EUA

f_shark.loc[(f_shark['ocean']=='USA'), 'area'].value_counts()

In [ ]:
f_shark.loc[(f_shark['ocean']=='USA') & (f_shark['area']=='New Mexico'), :]

In [ ]:
# Filtrando os incidentes no lado Pacífico dos EUA

usa_pcf = 'hawai|califo|alas|atoll|guam|washin|oreg'

filtro_usa_pcf = (f_shark['area'].apply(lambda pais: True if re.search(usa_pcf, str(pais).lower()) else False)) & (f_shark['country']=='USA')
f_shark.loc[filtro_usa_pcf, 'ocean'] = 'PACIFIC OCEAN'

In [ ]:
# Preenchendo os outros incidentes dos EUA com Atlântico

f_shark.loc[(f_shark['country']=='USA') & (f_shark['ocean'] != 'PACIFIC OCEAN'), 'ocean'] = 'ATLANTIC OCEAN'

###  Tratamento dados da Austrália

In [ ]:
# Verificando dados da Austrália

f_shark.loc[(f_shark['ocean']=='AUSTRALIA'), 'area'].value_counts()

In [ ]:
# Filtrando os incidentes no lado Índico da Austrália

aus_ind = 'western|south aus|keeling'

filtro_aus_ind = (f_shark['area'].apply(lambda pais: True if re.search(aus_ind, str(pais).lower()) else False)) & (f_shark['country']=='AUSTRALIA')
f_shark.loc[filtro_aus_ind, 'ocean'] = 'INDIAN OCEAN'

In [ ]:
# Preenchendo os outros incidentes da Austrália com Pacífico

f_shark.loc[(f_shark['country']=='AUSTRALIA') & (f_shark['ocean'] != 'INDIAN OCEAN'), 'ocean'] = 'PACIFIC OCEAN'

###  Tratamento dados da África do Sul

In [ ]:
# filtrando dados da África do Sul

f_shark.loc[(f_shark['ocean']=='SOUTH AFRICA'), 'area'].value_counts()

In [ ]:
# Filtrando os incidentes no lado Atlântico da África do Sul

afs_atl = 'western|atlant'

filtro_afs_atl = (f_shark['area'].apply(lambda pais: True if re.search(afs_atl, str(pais).lower()) else False)) & (f_shark['country']=='SOUTH AFRICA')
f_shark.loc[filtro_afs_atl, 'ocean'] = 'ATLANTIC OCEAN'

In [ ]:
# Preenchendo os outros incidentes da África do Sul com Índico

f_shark.loc[(f_shark['country']=='SOUTH AFRICA') & (f_shark['ocean'] != 'ATLANTIC OCEAN'), 'ocean'] = 'INDIAN OCEAN'

###  Tratamento dados do México

In [ ]:
# filtrando dados do México

f_shark.loc[(f_shark['ocean']=='MEXICO'), 'area'].value_counts()

In [ ]:
# Filtrando os incidentes no lado Atlântico do México

mex_atl = 'vera|quint|tamau|carib|taba|campe'

filtro_mex_atl = (f_shark['area'].apply(lambda pais: True if re.search(mex_atl, str(pais).lower()) else False)) & (f_shark['country']=='MEXICO')
f_shark.loc[filtro_mex_atl, 'ocean'] = 'ATLANTIC OCEAN'

In [ ]:
# Preenchendo os outros incidentes do México com Pacífico

f_shark.loc[(f_shark['country']=='MEXICO') & (f_shark['ocean'] != 'ATLANTIC OCEAN'), 'ocean'] = 'PACIFIC OCEAN'

###  Tratamento dados da Espanha

###  Tratamento dados do Panamá

In [ ]:
# Filtrando os incidentes no lado Pacífico do Panamá

pnm_pcf = 'gulf'

filtro_pnm_pcf = (f_shark['area'].apply(lambda pais: True if re.search(pnm_pcf, str(pais).lower()) else False)) & (f_shark['country']=='PANAMA')
f_shark.loc[filtro_pnm_pcf, 'area'] = 'PACIFIC OCEAN'

In [ ]:
# Preenchendo os outros incidentes do Panamá com Atlântico

f_shark.loc[(f_shark['country']=='PANAMA') & (f_shark['ocean'] != 'PACIFIC OCEAN'), 'ocean'] = 'ATLANTIC OCEAN'

###  Verificação

In [ ]:
# verificando 

paises = sorted(list(f_shark['ocean'].value_counts().index))
print(paises)

In [ ]:
# verificando linhas sem nenhuma informação do local

f_shark.loc[f_shark['ocean'].isnull(), :].shape

###  Completando dados remanescentes

In [ ]:
# completando dados remanescentes com Atlântico

resto = 'ATLANTIC OCEAN|PACIFIC OCEAN|INDIAN OCEAN'

filtro_resto = f_shark['ocean'].apply(lambda pais: False if re.search(resto, str(pais).upper()) else True)
f_shark.loc[filtro_resto, 'ocean'] = 'ATLANTIC OCEAN'

In [ ]:
f_shark['ocean'].value_counts()

## Tratamento de mês

def class_per(data):
    '''
    '''
    pad_mes = 'jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec'
    pad_est = 'fall|summer|winter|spring'
    
    if re.search(pad_mes, data.lower):
        per = 
        return re.findall(pad_)

In [ ]:
pat_month = 'jan|feb|mar|ap[r]?|may|jun|jul|aug|sep|oct|nov|dec|fall|summer|winter|spring'

f_shark['month'] = f_shark['date'].apply(lambda date: re.findall(pat_month, date.lower())[0] if re.search(pat_month, date.lower()) else 0)

In [ ]:
f_shark['month'].value_counts()

In [ ]:
f_shark['month'] = f_shark['date'].apply(lambda date: re.sub('\d{4}', '', date))

In [ ]:
f_shark['month']

In [ ]:
f_shark.loc[f_shark['month']== 0, 'date'].apply(lambda date: re.findall('\.(\d{2})\.', date))

In [ ]:
f_shark.loc[f_shark['month']==0, 'date'].apply(lambda date: re.sub('\d{4}', '', date)).value_counts()

## Tratamento de gênero

In [ ]:
# verificando coluna de gênero

f_shark['sex'].value_counts()

In [ ]:
# verificando linhas que não possuem nem informação de gênero e nem de nome

f_shark.loc[(f_shark['sex'].isnull()) & (f_shark['name'].isnull()), :]

In [ ]:
# verificando se há informações de incidentes com mulheres nos registros com coluna de gênero nula e substituindo

filtro_fem = f_shark.loc[(f_shark['sex'].isnull()) & ~(f_shark['name'].isnull()), 'name'].apply(lambda name: True if re.search('wom[ae]n', str(name)) else False)

f_shark.loc[(f_shark['sex'].isnull()) & ~(f_shark['name'].isnull()) & filtro_fem, 'sex'] = 'F'

In [ ]:
# ajustando valores da coluna de gênero

f_shark.loc[(f_shark['sex'] != 'M') & (f_shark['sex'] != 'F') & ~(f_shark['sex'].isnull()), 'sex'] = 'M'

In [ ]:
# preenchendo valores de gênero vazios com 'M'

f_shark.loc[(f_shark['sex'] != 'M') & (f_shark['sex'] != 'F') & (f_shark['sex'].isnull()), 'sex'] = 'M'

In [ ]:
f_shark['sex'].value_counts()

## Tratamento de fatalidade

In [ ]:
f_shark['fatal'].value_counts()

In [ ]:
# verificando dados que não tem informação de fatalidade e nem de ferimentos

f_shark.loc[(f_shark['injury'].isnull()) & (f_shark['fatal'].isnull()), :]

In [ ]:
# verificando dados de fatalidade que podem ser preenchidos com informações de 'injury'

f_shark.loc[~(f_shark['injury'].isnull()) & (f_shark['fatal'].isnull()), :]

In [ ]:
# substituindo dados de fatalidade nulos por indicativos de fatalidade indentificados em 'injury'

fatal_cond = 'fatal|death|mortem|dead|drown'

f_shark.loc[f_shark['fatal'].isnull(), 'fatal'] = f_shark.loc[f_shark['fatal'].isnull(), 'injury'].apply(lambda desc: 'Y' if re.search(fatal_cond, str(desc).lower()) else None)
f_shark.loc[f_shark['fatal'].isnull(), 'fatal'] = f_shark.loc[f_shark['fatal'].isnull(), 'species'].apply(lambda desc: 'Y' if re.search('death', str(desc).lower()) else None)

In [ ]:
# substituindo dados de fatalidade nulos por indicativos de não-fatalidade identificados em 'injury'

non_fatal_cond = 'no injury'

f_shark.loc[f_shark['fatal'].isnull(), 'fatal'] = f_shark.loc[f_shark['fatal'].isnull(), 'injury'].apply(lambda desc: 'N' if re.search(non_fatal_cond, str(desc).lower()) else None)

In [ ]:
# verificando dados após tratamento inicial

f_shark['fatal'].value_counts()

In [ ]:
# ajustando preenchimento de fatalidades e completando valores nulos com 'N'

f_shark.loc[(f_shark['fatal']=='y'), 'fatal'] = 'Y'
f_shark.loc[(f_shark['fatal'] != 'Y') & (f_shark['fatal'] != 'N'), 'fatal'] = 'N'

In [ ]:
# verificando dados pós-tratamento

f_shark['fatal'].value_counts()